In [ ]:
import pandas as pd
import numpy as np
import anndata as ad
import os
import sys
from collections import Counter
import gget
import scipy
from sklearn import preprocessing

import scanpy as sc
import scanpy.external as sce
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

# Load our data

In [ ]:
# fpath = "/scratch/indikar_root/indikar1/cstansbu/all_scHSC_run3/scanpy/anndata_distance.h5ad"
# fpath = "/scratch/indikar_root/indikar1/cstansbu/all_scHSC_run4/scanpy/anndata_distance.h5ad"
# fpath = "/scratch/indikar_root/indikar1/cstansbu/all_HSC/scanpy/anndata_distance.h5ad"
fpath = "/scratch/indikar_root/indikar1/cstansbu/all_HSC/scanpy/anndata_tabula_merged.h5ad"

adata = sc.read_h5ad(fpath)
adata

In [ ]:
# sample 
fraction = 0.75
data = sc.pp.subsample(adata, 
                       fraction=fraction, 
                       copy=True)
data

# Joint embedding 

In [ ]:
NPCA = 15
use_highly_variable=True
n_genes = 1500
knn = 31
impute = True
diffision = 2
magic_knn = 31
magic_solver = "approximate"
resolution = 0.1
min_distance = 1.0
spread = 1.0

# simple preprocessing
sc.pp.highly_variable_genes(data, n_top_genes=n_genes)

# imputation
if impute:
    sce.pp.magic(data, 
                 t=diffision, 
                 knn=magic_knn,
                 solver=magic_solver,
                 name_list='all_genes')

# PCA
sc.tl.pca(data, n_comps=NPCA, use_highly_variable=use_highly_variable)
sc.pl.pca_variance_ratio(data, n_pcs=30)

sc.pp.neighbors(data, n_neighbors=knn, n_pcs=NPCA)

# clustering
print("Clustering")
sc.tl.leiden(data, resolution=resolution,)
sc.tl.paga(data)
sc.pl.paga(data, plot=False)

sc.tl.umap(data, 
           min_dist=min_distance, 
           spread=spread, 
           init_pos='paga',)

# save the coordinates
data.obs['UMAP1'] = data.obsm['X_umap'][:, 0]
data.obs['UMAP2'] = data.obsm['X_umap'][:, 0]

sc.pl.umap(data, 
           color=['leiden', 
                  'set', 
                  'celltype', 
                  'cluster_name'], 
           ncols=2)

In [ ]:
sc.pl.umap(data, 
           color=['leiden', 
                  'organ_tissue'], 
           ncols=2)

In [ ]:
break